## run this on local environment

In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, Value, ClassLabel, Features


https://stackoverflow.com/questions/75510487/huggingface-trainer-k-fold-cross-validation

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
traindf = pd.read_excel('./data/Task-2/train.xlsx')
traindf['label'].replace(-1, 0, inplace=True)
trainds_features = Features({'text': Value(dtype='string', id = None), 'label': ClassLabel(num_classes=2 ,id=None)})
trainds = Dataset.from_dict(mapping={"text": traindf['text'].to_list(), 'label': traindf['label'].to_list()},
                            features=trainds_features)
trainds = trainds.train_test_split(train_size=0.8)
trainds

In [ ]:
from transformers import AutoTokenizer
checkpoint = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenized_sentences = tokenizer(trainds["text"])

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)
tokenized_datasets = trainds.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# samples = tokenized_datasets[:8]
# samples = {k: v for k, v in samples.items() if k not in ["text"]}
# [len(x) for x in samples["input_ids"]]

## modify epochs in training_args for real training

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
tokenized_datasets

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()